# Testing notebook

This notebook is meant for visualizing stuff and testing code. 

In [2]:
import numpy as np
import pandas as pd
import os
import warnings
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from contextlib import contextmanager
from typing import Generator
import demoji
warnings.filterwarnings("ignore", category=FutureWarning)

import spacy
from spacy.tokens import Doc
from spacy import displacy
from load_spacy import nlp

In [5]:

def _load_data(data_dir:str) -> pd.DataFrame:
    """Loads a directory of .jsonl files"""
    return pd.read_json(data_dir, lines=True)
    
    
pan_docs = _load_data("data/pan22/preprocessed/pan22_preprocessed.jsonl")["fullText"].to_list()


test_docs = [
    "It was Jane's car that got stolen last night.",
    "It is my boss who will send me home today.",
    "It is me who tom saw today.",
    "It was your friend Suzy I met!",
    "Was it the boy in the hat who saw you?",
    "It wasn't the dog in the tree that ate the apple."
]


docs = list(nlp.pipe(test_docs))

In [26]:

# [It (is/was) ]



for token in docs[0]:
    children = [child.text for child in token.children]
    print(token, token.head.text, children)
    if token.text in ["is", "was"]:
        if "it" in children:
            pass

It was []
was was ['It', 'car', '.']
Jane car ["'s"]
's Jane []
car was ['Jane', 'stolen']
that stolen []
got stolen []
stolen car ['that', 'got', 'night']
last night []
night stolen ['last']
. was []


In [28]:
displacy.render(docs[0])

In [29]:
displacy.render(nlp("it was cool that dad bought a car"))